In [ ]:
import os
path = os.getcwd() + '/oldjs'
new_path = os.getcwd() + '/newjs'
print(path, new_path)
files = [file for file in os.listdir(path)]
print(files)
for file in files:
    if '.js' not in file: continue
    with open(path + '/' + file, mode='r', encoding='utf-8') as of:
        print(file)
        old_f = of.read()
    with open(new_path + '/' + file, mode='w', encoding='utf-8') as nf:
        nf.write("/*create by author taric*/")
        nf.write(old_f)



In [ ]:
import requests
from bs4 import BeautifulSoup
from jinja2 import Environment, FileSystemLoader

# 首先，使用 requests 库获取公众号文章的 HTML 页面
url = 'https://mp.weixin.qq.com/s/La93IM5cMyvFcNehS8dIFw'  # 公众号文章的链接
response = requests.get(url)
html = response.text

# 然后，使用 BeautifulSoup 解析 HTML 页面，提取出需要的数据
soup = BeautifulSoup(html, 'html.parser')
title = soup.find('h1', class_='rich_media_title').text.strip()
date = soup.find('em', class_='rich_media_meta rich_media_meta_text').text.strip()
content = soup.find('div', class_='rich_media_content').prettify()

# 最后，使用 Jinja2 将数据填充到 HTML 模板中，生成最终的 HTML 文件
env = Environment(loader=FileSystemLoader('.'))
template = env.get_template('template.html')
output = template.render(title=title, date=date, content=content)

# 将最终的 HTML 文件保存到本地
with open('output.html', 'w', encoding='utf-8') as f:
    f.write(output)

In [ ]:
# coding: utf-8
import pdfkit
import os
import requests
from bs4 import BeautifulSoup

# 模板html,微信抓取到的html内容过多.
T_HTML = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="referrer" content="never">
    <meta name="referrer" content="no-referrer">
    <meta http-equiv="X-UA-Compatible" content="IE=edge">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Document</title>
    <style>{style}</style>
</head>
<body>
    {content}
</body>
</html>"""

# pdf的一些参数
PDF_OPTIONS = {
    'page-size': 'A4',
    'encoding': "UTF-8",
}


def getHtmlContent(url, proxies=None):
    '''
    获取html
    '''
    if proxies is None:
        proxies = {"http": None, "https": None}
    res = requests.get(url, proxies)
    res.encoding = 'utf-8'
    return res.text


def reHtmlTags(cnt_html):
    '''
    替换图片src、元素、删除元素
    '''
    # 替换图片标签属性
    cnt_html = cnt_html.replace(
        "data-src", "src").replace('style="visibility: hidden;"', "")
    soup = BeautifulSoup(cnt_html, 'html.parser')

    # 删除评论和投票的html标签
    if soup.iframe:
        soup.iframe.decompose()
    print(soup)
    # 用模板格式化
    comments = soup.findAll("img", {"class": "like_comment_pic"})
    styles = soup.find_all('style')
    meta_content = soup.find('div', id='meta_content')
    js_tags = soup.find('div', id='js_tags')
    content = soup.find('div', id='page-content')
    styles[0].string = styles[0].string + r'  img { width: 100%; }'
    fmt_html = T_HTML.format(style=styles[0].string, content=content)
    html = fmt_html.replace(comments[0].attrs['src'], '') if comments else fmt_html
    return html


def outFile(data, out_type):
    '''
    导出
    '''
    path = os.getcwd() + '/out.html'
    with open(path, 'w', encoding='utf-8') as f:
        f.write(data)


source = getHtmlContent('https://mp.weixin.qq.com/s/La93IM5cMyvFcNehS8dIFw')
html = reHtmlTags(source)
outFile(html, 'html')